In [1]:
%load_ext autoreload
%autoreload complete --print

In [64]:
from priority import PrioritySampler, normalize
import scipy
import numpy as np

Reloading 'priority.priority_sampler'.


In [93]:
# initialize a priority sampler
PS = PrioritySampler(kind="exp")

w = np.array([1,2,3,4,5,10])
size = 3

Reloading 'priority.priority_sampler'.


In [94]:
# Run many times and bootstrap statistics
n_runs = 1000
samples = np.array([PS.weighted_sample(w, size) for _ in range(n_runs)])
stats = scipy.stats.describe(samples, axis=0)

import pandas as pd
pd.options.plotting.backend = "plotly"

df = pd.DataFrame({
    'Actual': w,
    'Estimate mean': stats.mean,
    'Estimate sem': 1.96 * np.sqrt(stats.variance / stats.nobs)
}).reset_index()

import plotly.express as px
fig = px.scatter(
    df, x='Estimate mean', y='index', error_x='Estimate sem',
    labels = {"Estimate mean": "Value"}
).add_scatter(
    x=df['Actual'], y=df.index, mode='markers', name='w',
    marker=dict(symbol='diamond', size=10, opacity=0.6),
)

rng = np.random.default_rng()
bootstrap_results = scipy.stats.bootstrap((samples,), np.mean, confidence_level=0.95, random_state=rng)

df['m'] = np.mean(bootstrap_results.bootstrap_distribution, axis=1)
df['CIl'] = bootstrap_results.confidence_interval.low
df['CIh'] = bootstrap_results.confidence_interval.high
df['se'] = bootstrap_results.standard_error

display(df)
# Create error bars using the precomputed mean and sem
fig.add_scatter(
    x=df.m,
    y=df.index,
    error_x=dict(
        array=df.CIh - df.m,
        arrayminus=df.m - df.CIl
    ),
    mode='markers',
    showlegend=False
)
fig.update_layout(plot_bgcolor='white')
fig.show()

,index,Actual,Estimate mean,Estimate sem,m,CIl,CIh,se
0,0,1,0.949311,0.157981,0.950222,0.806059,1.122084,0.081253
1,1,2,2.080438,0.217077,2.079294,1.876190,2.303988,0.109150
2,2,3,3.127621,0.260163,3.127865,2.885963,3.405492,0.132603
3,3,4,4.123567,0.302750,4.122655,3.835666,4.447587,0.155161
4,4,5,5.105010,0.308853,5.105494,4.811674,5.429942,0.158380
5,5,10,9.903855,0.300059,9.903677,9.608086,10.196984,0.151603
